In [1]:
from pathlib import Path
import pandas as pd
from time import time

from sqlalchemy import create_engine, func
from sqlalchemy.orm import sessionmaker
from sqlalchemy.orm.exc import NoResultFound

from models import (Program, Block, Workout, Workout_set, Exercise, 
                    Log_workout, Log_set, Historic_pr, exercise_muscle)

In [2]:
engine = create_engine(f"sqlite:///data/db/gym_database.db")
Session = sessionmaker(bind=engine)
session = Session()

In [5]:
# record_to_del = session.query(Program).filter_by(program_id = 5).one()
# record_to_del
# session.delete(record_to_del)
# session.commit()
session.query(Block).all()

[<Block(id=2,desc=Micro 0 Gonzalo septiembre2021,program=Macro Pisano)>,
 <Block(id=3,desc=Micro1 Gonzalo septiembre2021,program=Macro Pisano)>,
 <Block(id=4,desc=Micro2 Gonzalo septiembre2021,program=Macro Pisano)>,
 <Block(id=5,desc=Micro3 Gonzalo octubre2021,program=Macro Pisano)>,
 <Block(id=6,desc=Micro4 Gonzalo octubre2021,program=Macro Pisano)>,
 <Block(id=7,desc=Micro5 Gonzalo octubre2021,program=Macro Pisano)>,
 <Block(id=8,desc=Micro6 Gonzalo octubre2021,program=Macro Pisano)>,
 <Block(id=9,desc=Micro5 Gonzale,program=Macro Pisano)>]

In [4]:
###### Change exercise name ######
# 1st. Get exercises IDs to change in all necessary tables
exer_to_change =  session.query(Exercise).filter(Exercise.exercise_desc.like("dominadas libres%")).all()
print(exer_to_change)
ids_to_change = [i.exercise_id for i in exer_to_change]
# 2nd. Get "valid" exercise ID to replace
valid_exer = session.query(Exercise).filter(Exercise.exercise_desc.like("dominadas")).one()
print(valid_exer)

# 3rd. Get all affected records in necessary tables
wods_to_change = session.query(Workout_set).filter(Workout_set.exercise_id.in_(ids_to_change)).all()
prs_to_change = session.query(Historic_pr).filter(Historic_pr.exercise_id.in_(ids_to_change)).all()
ex_mus_to_change = session.query(exercise_muscle).filter(exercise_muscle.c.exercise_id.in_(ids_to_change)).all()
# print(wods_to_change)

# 4th. Update to "valid" ID in all tables
for wod in wods_to_change:
    wod.exercise_id = valid_exer.exercise_id
    print(wod)
for pr in prs_to_change:
    pr.exercise_id = valid_exer.exercise_id
    print(pr)
for ex_mus in ex_mus_to_change:
    ex_mus.exercise_id = valid_exer.exercise_id
    print(ex_mus)

# 5th. Finally, drop the now invalid name records
for record in exer_to_change:
    session.delete(record)

# print(wods_to_change)

[<Exercise(id=11,desc=dominadas libres)>, <Exercise(id=73,desc=dominadas libres )>]
<Exercise(id=2,desc=dominadas)>


In [5]:
session.query(Exercise).filter(Exercise.exercise_id.in_([11, 73])).all()
# exercise_muscle

[]

In [ ]:
[i.exercise_id for i in exer_to_change]
valid_exer.exercise_id